In [1]:
# %%
# # Heart Disease Prediction: CardioTabNet Implementation
# This notebook will guide you through:
# 1. Data loading and exploration
# 2. Preprocessing
# 3. Baseline model evaluation
# 4. CardioTabNet model definition, training, and evaluation
# 5. Exporting the trained model for FastAPI deployment

# %%
# 1. Imports and Setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [18]:
!pip install pandas numpy scikit-learn torch xgboost joblib


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [22]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [23]:
# 2. Load Data
file_path = r"C:/Users/user/OneDrive/Desktop/smirthi/JN/kaggle/heart.csv"
df = pd.read_csv(file_path)
print(df.shape)
df.head()

(1025, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [24]:
print(df.isnull().sum())



age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [25]:
# If no missing values, proceed; else, impute appropriately
# Encode target
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])

In [26]:

# Features and target
y = df['target'].values
n_features = [c for c in df.columns if c != 'target']
X = df[n_features].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# %%
# 4. Baseline: XGBoost (for comparison)

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 1.3 MB/s eta 0:01:56
   ---------------------------------------- 0.8/150.0 MB 1.2 MB/s eta 0:02:01
   ---------------------------------------- 1.0/150.0 MB 1.0 MB/s eta 0:02:26
   ---------------------------------------- 1.0/150.0 MB 1.0 MB/s eta 0:02:26
   ---------------------------------------- 1.3/150.0 MB 919.0 kB/s eta 0:02:42
   ---------------------------------------- 1.3/150.0 MB 919.0 kB/s eta 0:02:42
   ---------------------------------------- 1.3/150.0 MB 919.0 kB/s eta 0:02:42
   ---------------------------------------- 1.3/150.0 MB 919.0 kB/s eta 0:02:42
   ---------------------------------------- 1.6/150.0 MB 630.8 kB/s eta 0:03:56
   -


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [31]:
from xgboost import XGBClassifier
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("XGBoost AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))


XGBoost Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       105

    accuracy                           1.00       205
   macro avg       1.00      1.00      1.00       205
weighted avg       1.00      1.00      1.00       205

XGBoost AUC: 1.0


In [32]:
pip install optuna


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [36]:
# 5. CardioTabNet Dataset & Dataloader
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TabularDataset(X_train, y_train)
test_ds  = TabularDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

In [44]:
# 6. CardioTabNet Model Definition
class CardioTabNet(nn.Module):
    def __init__(self, input_dim, embed_dim=32, num_heads=4, num_layers=2, mlp_hidden=64):
        super().__init__()
        # Embedding layers per feature (optional: treat each feature separately)
        self.fc_embed = nn.Linear(input_dim, embed_dim)
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=mlp_hidden, dropout=0.1,batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Classifier head
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(mlp_hidden, 2)
        )

    def forward(self, x):
        x = self.fc_embed(x)            # [batch, embed_dim]
        x = self.transformer(x)         # [batch, embed_dim] (no unsqueeze!)
        logits = self.classifier(x)
        return logits


In [45]:
# Instantiate model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CardioTabNet(input_dim=X_train.shape[1]).to(device)

# Define optimizer and loss separately
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()



In [46]:
# %%
# 7. Training Loop
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(loader.dataset)

In [48]:
# 8. Evaluation Function
def evaluate(model, loader):
    model.eval()
    preds, trues, probs = [], [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            logits = model(X_batch)
            prob = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
            pred = np.argmax(logits.cpu().numpy(), axis=1)
            preds.extend(pred)
            trues.extend(y_batch.numpy())
            probs.extend(prob)
    acc = accuracy_score(trues, preds)
    auc = roc_auc_score(trues, probs)
    return acc, auc


In [49]:
# %%
# 9. Run Training & Evaluation
n_epochs = 30
best_auc = 0
for epoch in range(1, n_epochs+1):
    loss = train(model, train_loader, optimizer, criterion)
    acc, auc = evaluate(model, test_loader)
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), 'cardio_tabnet_best.pt')
    print(f"Epoch {epoch:02d}: loss={loss:.4f}, test_acc={acc:.4f}, test_auc={auc:.4f}")

Epoch 01: loss=0.6557, test_acc=0.7415, test_auc=0.8669
Epoch 02: loss=0.4894, test_acc=0.7951, test_auc=0.8885
Epoch 03: loss=0.3986, test_acc=0.8390, test_auc=0.9129
Epoch 04: loss=0.3392, test_acc=0.8537, test_auc=0.9297
Epoch 05: loss=0.3121, test_acc=0.8537, test_auc=0.9372
Epoch 06: loss=0.2971, test_acc=0.8634, test_auc=0.9442
Epoch 07: loss=0.2813, test_acc=0.8683, test_auc=0.9480
Epoch 08: loss=0.2492, test_acc=0.8634, test_auc=0.9531
Epoch 09: loss=0.2279, test_acc=0.8976, test_auc=0.9565
Epoch 10: loss=0.2220, test_acc=0.8634, test_auc=0.9645
Epoch 11: loss=0.2045, test_acc=0.8927, test_auc=0.9596
Epoch 12: loss=0.1907, test_acc=0.8927, test_auc=0.9728
Epoch 13: loss=0.2037, test_acc=0.9220, test_auc=0.9718
Epoch 14: loss=0.1762, test_acc=0.9122, test_auc=0.9707
Epoch 15: loss=0.1666, test_acc=0.9073, test_auc=0.9835
Epoch 16: loss=0.1381, test_acc=0.9220, test_auc=0.9787
Epoch 17: loss=0.1321, test_acc=0.9463, test_auc=0.9839
Epoch 18: loss=0.1248, test_acc=0.9220, test_auc

In [53]:
# %%
# 10. Load Best Model & Final Metrics
model.load_state_dict(torch.load('cardio_tabnet_best.pt'))
final_acc, final_auc = evaluate(model, test_loader)
print(f"Best saved model -- Accuracy: {final_acc:.4f}, AUC: {final_auc:.4f}")
with torch.no_grad():
    test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    test_logits = model(test_tensor)  # Get logits from the model
    test_preds = np.argmax(torch.softmax(test_logits, dim=1).detach().cpu().numpy(), axis=1)
    print(classification_report(y_test, test_preds))



Best saved model -- Accuracy: 0.9805, AUC: 0.9988
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       100
           1       0.98      0.99      0.99       105

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205



In [54]:
# %%
# 11. Export for FastAPI Deployment
# Save the scaler and model
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']